In [62]:
!pip install SPARQLWrapper elasticsearch pyhive thrift sentence_transformers thrift_sasl


  Preparing metadata (setup.py) ... done
  Created wheel for pure-sasl: filename=pure_sasl-0.6.2-py3-none-any.whl size=11427 sha256=4dd1118be8fbbf1800747e715e4582055e6b9967e04f9c6f8f2448a9abd46e09
  Stored in directory: /home/jovyan/.cache/pip/wheels/af/5e/ca/57ff2c5801d038e3d8b227a4fb492cd84e43a535d64a06f3f2
Successfully built pure-sasl


In [63]:
import json
from elasticsearch import Elasticsearch
from pyhive import hive
import pandas as pd
from sentence_transformers import SentenceTransformer
import requests
from SPARQLWrapper import SPARQLWrapper, JSON


# Consulta GraphDB

In [72]:

# Define the endpoint URL to the GraphDB repository, using 'graphdb' as the hostname
endpoint_url = "http://graphdb:7200/repositories/Hola"  # Replace 'your-repo-name' with your actual repository name

# Create a SPARQLWrapper instance
sparql = SPARQLWrapper(endpoint_url)

# Set the authentication credentials
sparql.setCredentials("MANOLO", "ALONSO")  # Username and password for authentication

# Define the SPARQL query
sparql_query = """
PREFIX ont: <http://upm.es/ontology#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?database ?dbname ?host ?port ?user ?password
WHERE {
    ?database rdf:type ont:Database .
    OPTIONAL { ?database ont:dbname ?dbname . }
    OPTIONAL { ?database ont:host ?host . }
    OPTIONAL { ?database ont:port ?port . }
    OPTIONAL { ?database ont:user ?user . }
    OPTIONAL { ?database ont:password ?password . }
}
ORDER BY ?database
"""

# Set the SPARQL query and the return format (JSON)
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)

# Execute the query and get the results
try:
    results = sparql.query().convert()

except Exception as e:
    print("An error occurred:", e)


In [73]:

# Initialize lists to hold results for Hive and Elastic
hive_db = []
elastic_db = []

# Iterate through the results and filter based on the database type
for result in results['results']['bindings']:
    database_uri = result['database']['value']
    
    if 'Elastic' in database_uri:  # Filter for Elastic databases
        elastic_db.append({
            'dbname': result.get('dbname', {}).get('value', None),
            'host': result.get('host', {}).get('value', None),
            'port': result.get('port', {}).get('value', None),
        })
    elif 'Hive' in database_uri:  # Filter for Hive databases
        hive_db.append({
            'dbname': result.get('dbname', {}).get('value', None),
            'host': result.get('host', {}).get('value', None),
            'port': result.get('port', {}).get('value', None),
            'user': result.get('user', {}).get('value', None),
        })
        
        

In [67]:
# import requests

# # GraphDB SPARQL endpoint - use the service name 'graphdb' instead of 'localhost'
# url = "http://graphdb:7200/repositories/Hola"
# username = "MANOLO"
# password = "ALONSO"

# # Define the SPARQL query
# sparql_query = """
# PREFIX ont: <http://upm.es/ontology#>
# PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
# PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
# PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

# SELECT DISTINCT ?database ?dbname ?host ?port ?user ?password
# WHERE {
#     ?database rdf:type ont:Database .
#     OPTIONAL { ?database ont:dbname ?dbname . }
#     OPTIONAL { ?database ont:host ?host . }
#     OPTIONAL { ?database ont:port ?port . }
#     OPTIONAL { ?database ont:user ?user . }
#     OPTIONAL { ?database ont:password ?password . }
# }
# ORDER BY ?database
# """

# # Set headers for the request
# headers = {
#     "Content-Type": "application/sparql-query",
#     "Accept": "application/sparql-results+json"
# }

# # Send the POST request with the SPARQL query
# response = requests.post(url, data=sparql_query, headers=headers, auth=(username, password))

# # Check the response
# if response.status_code == 200:
#     results = response.json()
#     for result in results["results"]["bindings"]:
#         print({
#             "database": result.get("database", {}).get("value"),
#             "dbname": result.get("dbname", {}).get("value"),
#             "host": result.get("host", {}).get("value"),
#             "port": result.get("port", {}).get("value"),
#             "user": result.get("user", {}).get("value"),
#             "password": result.get("password", {}).get("value"),
#         })
# else:
#     print(f"Error: {response.status_code}, {response.text}")


# Consulta hive

In [68]:
# Conexión a HiveServer2
conn = hive.Connection(host=hive_db[0]["host"], port=hive_db[0]["port"], 
                       username=hive_db[0]["user"], database=hive_db[0]["dbname"])

# Crear un cursor
cursor = conn.cursor()
cursor = conn.cursor()

# Ejecutar una consulta
cursor.execute("""SELECT DISTINCT d.driverid, d.forename, d.surname
FROM races r
JOIN laps l ON r.raceid = l.raceid
JOIN drivers d ON l.driverid = d.driverid
WHERE r.name = 'Malaysian Grand Prix'
  AND l.lap = (SELECT MAX(lap) FROM laps WHERE raceid = r.raceid)
  AND l.position = 1""")

# Obtener los resultados
hive_results = pd.DataFrame(cursor.fetchall(), columns=['driverid', 'forename', 'surname'])

display(hive_results)

# Cerrar la conexión
cursor.close()
conn.close()

,driverid,forename,surname
0,1,Lewis,Hamilton
1,4,Fernando,Alonso
2,8,Kimi,Räikkönen
3,18,Jenson,Button
4,20,Sebastian,Vettel
5,21,Giancarlo,Fisichella
6,23,Ralf,Schumacher
7,30,Michael,Schumacher
8,56,Eddie,Irvine
9,817,Daniel,Ricciardo


# Consulta elasticsearch

In [74]:
es = Elasticsearch( f"http://{elastic_db[0]['host']}:{elastic_db[0]['port']}")

index_name = elastic_db[0]["dbname"]

model = SentenceTransformer('intfloat/multilingual-e5-large')

In [75]:
query_sentence = "query: pilotos que corrieron en le mans y en indianapolis"
query_vector = model.encode([query_sentence])[0]

parameters = {
    "field":"wikipedia_description_vector",
    "query_vector": query_vector,
    "k": 100,
    "num_candidates":100
}

res = es.search(index=index_name, knn=parameters, size=100)
es_results = pd.DataFrame(
    [{"driverid": x["_source"]["driverId"], "wikipedia_description": x["_source"]["wikipedia_description"]}
     for x in res['hits']['hits']])
display(es_results)

,driverid,wikipedia_description
0,296,François Migault (4 December 1944 – 29 January...
1,7,Sébastien Olivier Bourdais[1] (French pronunci...
2,513,"Eddie Johnson (February 10, 1919 – June 30, 19..."
3,212,Héctor Alonso Rebaque (born 5 February 1956)[1...
4,615,"Fred Levon Agabashian[1][2] (August 21, 1913 –..."
...,...,...
95,831,Luiz Felipe de Oliveira Nasr (born 21 August 1...
96,66,Allan McNish (born 29 December 1969) is a Brit...
97,197,Jean-Pierre Jacques Jarier[1] (born 10 July 19...
98,627,Louis Claude Rosier (5 November 1905 in Chapde...


# Join dataframe

In [9]:
# Realizar el join por driverid
combined_results = pd.merge(hive_results, es_results, on="driverid", how="inner")

display(combined_results)


,driverid,forename,surname,wikipedia_description
0,4,Fernando,Alonso,Fernando Alonso Díaz (Spanish pronunciation: [...
1,56,Eddie,Irvine,"Edmund ""Eddie"" Irvine Jr. (/ˈɜːrvaɪn/; born 10..."
